In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [49]:
!pip install eli5
!pip install --upgrade tables

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [50]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [51]:
ls

data/  day2_visualisation.ipynb


In [52]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [53]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [54]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [55]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [56]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [57]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df['price_currency']!= 'EUR']

In [59]:
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

FEATURES

In [60]:
df['param_color'].factorize()

(array([-1, -1, -1, ..., -1, -1, -1]),
 Index(['Czarny', 'Srebrny', 'Złoty', 'Bordowy', 'Biały', 'Szary', 'Beżowy',
        'Zielony', 'Fioletowy', 'Niebieski', 'Inny kolor', 'Brązowy', 'Żółty'],
       dtype='object'))

In [0]:
SUFFIX = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX] = factorized_values

In [62]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl__cat', 'param_stan__cat', 'param_wersja__cat',
       'param_emisja-co2__cat', 'param_body-type__cat', 'param_matowy__cat',
       'param_bezwypadkowy__cat', 'param_akryl-(niemetalizowany)__cat',
       'param_monthly-payment-value__cat', 'car_id__cat'],
      dtype='object', length=309)

In [68]:
cat_feats = [x for x in df.columns if SUFFIX in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [72]:
X = df[cat_feats].values
y = df['price_value']

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [76]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m).fit(X, y, random_state=666)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2574 ± 0.0088,param_napęd__cat
0.2034 ± 0.0056,param_faktura-vat__cat
0.1927 ± 0.0042,param_stan__cat
0.1405 ± 0.0037,param_rok-produkcji__cat
0.0657 ± 0.0020,param_moc__cat
0.0422 ± 0.0011,param_skrzynia-biegów__cat
0.0419 ± 0.0003,feature_kamera-cofania__cat
0.0279 ± 0.0024,param_marka-pojazdu__cat
0.0188 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0003,feature_bluetooth__cat
